## Calculate subindicator B

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
ee.Initialize(project="ee-dfgm2006")
import pygaul
import pandas as pd
from pathlib import Path

import component.parameter.module_parameter as param
from component.scripts.gee import reduce_regions
from component.scripts.scripts import get_b_years, map_matrix_to_dict, read_from_csv, get_sub_b_items, get_reporting_years, parse_to_year
from component.scripts import sub_b as sub_b

from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# If using outsite SEPAL, manually set a project

In [3]:
# Here I'm using pygaul to get the AOI based on the name
# but you can use any feature collection from Earth Engine
# just replace the aoi with your feature collection.

aoi_name = "Risaralda"
aoi = pygaul.Items(aoi_name)

In [4]:
# Read the default land cover remapping table and convert it into a dictionary
default_map_matrix = map_matrix_to_dict(param.LC_MAP_MATRIX)

# Set the defauult transition matrix path
default_transition_matrix_path = param.TRANSITION_MATRIX_FILE

In [5]:
# For SUB_B indicator, we need to set the following structure
sub_b_year = {
    "baseline": {
        "base": {
            "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2000",
            "year": 2000,
        },
        "report": {
            "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2015",
            "year": 2015,
        },
    },
    # And the reporting year
    2: {"asset": "users/amitghosh/sdg_module/esa/cci_landcover/2018", "year": 2018},
}

# 
years = get_b_years(sub_b_year)
years

[({'asset': 'users/amitghosh/sdg_module/esa/cci_landcover/2000', 'year': 2000},
  {'asset': 'users/amitghosh/sdg_module/esa/cci_landcover/2015', 'year': 2015},
  {'asset': 'users/amitghosh/sdg_module/esa/cci_landcover/2018',
   'year': 2018})]

In [6]:
process = ee.FeatureCollection([
    ee.Feature(
        None, 
        reduce_regions(
            aoi,
            remap_matrix=default_map_matrix,
            rsa=False,
            dem=param.DEM_DEFAULT, 
            lc_years= year,
            transition_matrix=default_transition_matrix_path
        )
    ).set("process_id", "_".join([str(y["year"]) for y in year]))
 for year in years
])

In [7]:
task_name = "Risaralda_sub_b"

In [8]:
task = ee.batch.Export.table.toDrive(
    **{
        "collection": process,
        "description": task_name,
        "fileFormat": "CSV",
    }
)
task.start()

# Read, process, and create report tables

In [ ]:
# Manually check your earth engine task status, once the task is completed, run the next cell
# according to the task name you have set above.

In [ ]:

results_file_path = Path().home()/ "downloads"/ "Risaralda_sub_b.csv"
report_folder = Path().home() / f"downloads/sub_b_report/{aoi_name}"
report_folder.mkdir(parents=True, exist_ok=True)

output_name = str(
    Path(report_folder, f"Sub_b_{aoi_name}.xlsx")
)

In [ ]:
# This line will read the results from the CSV file and parse it to a dictionary
dict_results = read_from_csv(results_file_path)
dict_results

In [ ]:
sub_b_reports = []
reporting_years_sub_b = get_reporting_years(sub_b_year, "sub_b")
_, sub_b_years = get_sub_b_items(reporting_years_sub_b)

for year in sub_b_years:
    print(f"Reporting {year} for sub_b")
    # Get year label for the report
    parsed_df = parse_to_year(dict_results, year)
    sub_b_reports.append(
        sub_b.get_reports(
            parsed_df, 
            year, 
            geo_area_name = "", 
            ref_area = "", 
            source_detail = "", 
            transition_matrix = default_transition_matrix_path
        ))

In [ ]:
# sub b reports
er_mtn_dgrp_df = pd.concat([report[0] for report in sub_b_reports])
er_mtn_dgda_df = pd.concat([report[1] for report in sub_b_reports])

In [ ]:
# This will create the excel file with the reports

with pd.ExcelWriter(output_name) as writer:
    er_mtn_dgda_df.to_excel(writer, sheet_name="Table4_ER_MTN_DGRDA", index=False)
    er_mtn_dgrp_df.to_excel(writer, sheet_name="Table5_ER_MTN_DGRDP", index=False)
    
    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]
        for col in worksheet.columns:
            max_length = 0
            column = col[0]
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = max(max_length, len(str(column.value))) + 4
            worksheet.column_dimensions[get_column_letter(column.column)].width = (
                adjusted_width
            )

            # Align "obs_value" column to the right
            if "OBS" in column.value:
                for cell in col:
                    cell.alignment = Alignment(horizontal="right")